# Übung 4: mehr binäre Klassifikation und Challenge

## Aufgabe 1: Titanic

![](images/titanic.jpg)

#### Wir möchten anhand von Daten vorhersagen ob eine Person den Untergang der Titanic überlebt hat. Dafür nehmen wir an einem Challenge auf der Plattform "Kaggle" teil.

1. Registrieren Sie sich auf https://www.kaggle.com/c/titanic und lesen Sie die Einführung auf https://www.kaggle.com/competitions/titanic/overview
2. Machen Sie sich mit den Daten auf https://www.kaggle.com/competitions/titanic/data vertraut
3. Laden Sie die Dateien 
    - Trainingsdaten(train.csv)
    - Testdaten (test.csv)

4. Selektieren Sie sinnvolle Features und teilen Sie die Features in die Kategorien: Numerisch, Ordinal und Nominal auf
3. Bauen Sie jeweils eine Data Pipeline zur Verarbeitung der jeweiligen Features.
4. Benutzen Sie die scitkit-learn Klasse `ColumnTransformer` um die Pipelines zu einer Gesamtpipeline wieder zusammen zu führen.
6. Wenden Sie die Gesamtpipeline auf die Trainingsdaten an 
7. Trainieren Sie mit den Trainingsdaten auf mehreren Klassifiziereren (z.B. `SGDClassifier, DecisionTreeClassifier, SVMClassifier, RandomForestClassifier`) 
8. Evalusieren Sie mit verschiedenen Metriken welcher der beste Klassifizierer ist
9. Schreiben Sie Ihre Ergebnisse in eine CSV Datei. Orientieren Sie sich an dem von Kaggle erwarteten Format `gender_submission.csv`.
10. Senden Sie die Daten ab und sehen Sie wie gut Sie im vergleich zu den anderen Teilnehmenden abschneiden.


In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    confusion_matrix,
    f1_score,
)
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC

In [2]:
titanic_train = pd.read_csv("data/titanic/train.csv")
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
ordinal_features = ["Sex"]
nominal_features = ["Embarked", "Cabin"]
numeric_features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]

In [9]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)
nominal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoding", OneHotEncoder(handle_unknown="ignore")),
    ]
)
ordinal_transfomer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal_encoding", OrdinalEncoder()),
    ]
)

# Gesamtpipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat_nominal", nominal_transformer, nominal_features),
        ("cat_ordinal", ordinal_transfomer, ordinal_features),
    ]
)

set_config(display="diagram")
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('cat_nominal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoding',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Embarked', 'Cabin']),
                                ('cat_ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal_encoding',
                                                  OrdinalEncoder())]),
                                 ['Sex'])])

In [10]:
X = titanic_train.drop("Survived", axis=1)
y = titanic_train[["Survived"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)

y_train_prepared = y_train.copy()  # no pre-processing necessary

In [11]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_train_prepared

<668x121 sparse matrix of type '<class 'numpy.float64'>'
	with 5105 stored elements in Compressed Sparse Row format>

In [35]:
X_test_prepared = preprocessor.transform(X_test)

classifier = {
    "Neural Network": MLPClassifier(),
    "Support Vector Machine": SVC(),
    "RandomForest": RandomForestClassifier(),
    "SGD": SGDClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
}

for name, c in classifier.items():
    print(name)
    clf = c
    clf.fit(X=X_train_prepared, y=y_train_prepared)

    predicted = clf.predict(X_test_prepared)

    precision = precision_score(y_pred=predicted, y_true=y_test)
    recall = recall_score(y_pred=predicted, y_true=y_test)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print("\n")

Neural Network


/usr/lib64/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib64/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/lib64/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_22625/1291052323.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X=X_train_prepared, y=y_train_prepare

precision: 0.7746478873239436
recall: 0.6395348837209303


Support Vector Machine
precision: 0.7307692307692307
recall: 0.6627906976744186


RandomForest
precision: 0.7349397590361446
recall: 0.7093023255813954


SGD
precision: 0.6521739130434783
recall: 0.6976744186046512


GradientBoosting
precision: 0.7971014492753623
recall: 0.6395348837209303




/usr/lib64/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib64/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
